In [ ]:
import os
import time
import config
from datetime import datetime
from log_utils import (
    read_log_messages, normalize_template, normalize_template_v1, normalize_template_v2,
    save_templates, extract_last_template_from_history
)
from ollama_utils import start_ollama_server, stop_ollama_server
from autogen import AssistantAgent
from codecarbon import EmissionsTracker
from evaluation import evaluate_and_save

In [ ]:
# --- Configuration ---
llm_config = config.LLM_CONFIG
task = config.TASK_PROMPT
sys_prompt_few_shot_log_parser = config.SYS_MSG_LOG_PARSER_FEW_SHOT
sys_prompt_few_shot_critic = config.SYS_MSG_CRITIC_FEW_SHOT
sys_prompt_zero_shot_log_parser = config.SYS_MSG_LOG_PARSER_ZERO_SHOT
sys_prompt_zero_shot_critic = config.SYS_MSG_CRITIC_ZERO_SHOT

LOG_DIR = config.LOG_DIR
RESULT_DIR = config.RESULT_DIR
os.makedirs(RESULT_DIR, exist_ok=True)

# change this for different designs, i.e., "DA-zero"
DESIGN = "DA-few"
model = llm_config["config_list"][0]["model"].replace(":", "-")
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
project_name = DESIGN.capitalize()
exp_name = f"{project_name}_{model}_{timestamp}"

input_log_file = "HDFS_200_sampled.log"
log_path = os.path.join(LOG_DIR, input_log_file)
ground_truth_file_path = os.path.join(LOG_DIR, "HDFS_200_sampled_log_structured.csv")


In [ ]:
# --- Agent Creation ---
def create_log_parser_agent(llm_config, sys_prompt):
    return AssistantAgent(
        name="log_parser_agent",
        system_message=sys_prompt,
        description="Analyze the log message in order to determine the corresponding template.",
        llm_config=llm_config,
        human_input_mode="NEVER",
    )

def create_critic_agent(llm_config, sys_prompt):
    return AssistantAgent(
        name="critic_agent",
        system_message=sys_prompt,
        llm_config=llm_config,
        human_input_mode="NEVER",
    )

In [ ]:
# --- Log Reading ---
logs = read_log_messages(log_path)

In [ ]:
# --- Inference with Two Agents and Emissions Tracking ---
def run_two_agent_inference(logs, llm_config, sys_prompt_log_parser, sys_prompt_critic, task, exp_name, result_dir):
    last_templates = []
    tracker = EmissionsTracker(project_name=exp_name, output_dir=result_dir, save_to_file=True)
    tracker.start()
    try:
        log_parser = create_log_parser_agent(llm_config, sys_prompt_log_parser)
        critic = create_critic_agent(llm_config, sys_prompt_critic)
        for i, log_message in enumerate(logs):
            res = critic.initiate_chat(
                recipient=log_parser,
                message=task + log_message,
                max_turns=2,
                summary_method="last_msg"
            )
            history = res.chat_history
            last_template = extract_last_template_from_history(history)
            print(f"\n--- Processing log {i} ---")
            print(f"Last template: {last_template}")
            if last_template:
                last_templates.append(last_template)
            else:
                last_templates.append("NONE")
                print(f"No valid template extracted from log_parser_agent for log {i}")
    finally:
        emissions = tracker.stop()
    print(f"Emissions: {emissions} kg CO2")
    return last_templates


In [ ]:
# --- Ollama Server Management ---
#proc = start_ollama_server()
time.sleep(5)  # Give it some time to initialize

last_templates = run_two_agent_inference(logs, llm_config, sys_prompt_few_shot_log_parser, sys_prompt_few_shot_critic, task, exp_name, RESULT_DIR)
save_templates(last_templates, llm_config, DESIGN, RESULT_DIR)

#stop_ollama_server(proc)

In [ ]:
# --- Evaluation ---
results = evaluate_and_save(normalize_template, last_templates, ground_truth_file_path, exp_name)
results_v1 = evaluate_and_save(normalize_template_v1, last_templates, ground_truth_file_path, exp_name)
results_v2 = evaluate_and_save(normalize_template_v2, last_templates, ground_truth_file_path, exp_name)


In [ ]:
#print("Results:", results)